
# Fundamentos de Deep Learning

Andres Felipe Osorio Henker

Universidad de Antioquia

Sistema de Predicción de futuras fallas en servidores de data centers basada en la telemetría

Exploración de los datos

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import linear_model
from wordcloud import WordCloud
import tensorflow as tf
from tensorflow.keras import utils
tf.__version__
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras import regularizers, optimizers
from google.colab import drive
import tarfile

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#with tarfile.open("/content/drive/My Drive/dataset/PdM_telemetry.tar.xz", "r:xz") as archivo:
    # Extraer todos los archivos en la carpeta actual
#    archivo.extract("PdM_telemetry.csv", "/content/drive/My Drive/dataset/")

#print(telemetry_data.shape)
#telemetry_data.head(10)

In [73]:
telemetry_data = pd.read_csv('/content/drive/My Drive/dataset/PdM_telemetry.csv')
print(telemetry_data.shape)
telemetry_data.head(3)

(876100, 6)


,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847


In [74]:
telemetry_data.groupby('machineID').count()

,datetime,volt,rotate,pressure,vibration
machineID,,,,,
1,8761,8761,8761,8761,8761
2,8761,8761,8761,8761,8761
3,8761,8761,8761,8761,8761
4,8761,8761,8761,8761,8761
5,8761,8761,8761,8761,8761
...,...,...,...,...,...
96,8761,8761,8761,8761,8761
97,8761,8761,8761,8761,8761
98,8761,8761,8761,8761,8761


In [75]:
telemetry_data_one = telemetry_data.loc[telemetry_data["machineID"] == 1]
telemetry_data_one.groupby('machineID').count()

,datetime,volt,rotate,pressure,vibration
machineID,,,,,
1,8761,8761,8761,8761,8761


In [76]:
error_data = pd.read_csv('/content/drive/My Drive/dataset/PdM_failures.csv')
print(error_data.shape)
error_data.head(3)

(761, 3)


,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2


In [77]:
error_data = error_data.drop_duplicates()
error_data

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4
...,...,...,...
756,2015-11-29 06:00:00,99,comp3
757,2015-12-14 06:00:00,99,comp4
758,2015-02-12 06:00:00,100,comp1
759,2015-09-10 06:00:00,100,comp1


In [78]:
datos=pd.merge(telemetry_data_one, error_data, on=['datetime', 'machineID'], how="left")
datos=datos.drop(["machineID"], axis = 1)
datos.fillna(0, inplace=True)
datos.groupby('failure').count()

,datetime,volt,rotate,pressure,vibration
failure,,,,,
0,8754,8754,8754,8754,8754
comp1,1,1,1,1,1
comp2,2,2,2,2,2
comp4,4,4,4,4,4


In [79]:
datos['datetime'] = pd.to_datetime(datos['datetime'] )
datos = datos.set_index('datetime')

In [80]:
datos["failure"] = np.where(datos["failure"] != 0, 1, datos["failure"])

In [81]:
datos["failure"].value_counts()

failure
0    8754
1       7
Name: count, dtype: int64

In [82]:
datos.iloc[:, :4]

,volt,rotate,pressure,vibration
datetime,,,,
2015-01-01 06:00:00,176.217853,418.504078,113.077935,45.087686
2015-01-01 07:00:00,162.879223,402.747490,95.460525,43.413973
2015-01-01 08:00:00,170.989902,527.349825,75.237905,34.178847
2015-01-01 09:00:00,162.462833,346.149335,109.248561,41.122144
2015-01-01 10:00:00,157.610021,435.376873,111.886648,25.990511
...,...,...,...,...
2016-01-01 02:00:00,153.724780,450.254337,85.197443,43.698137
2016-01-01 03:00:00,172.418099,375.701907,95.568265,42.111661
2016-01-01 04:00:00,174.588145,460.505353,95.483913,33.293572


In [84]:
datos.iloc[:, [-1]]

,failure
datetime,
2015-01-01 06:00:00,0
2015-01-01 07:00:00,0
2015-01-01 08:00:00,0
2015-01-01 09:00:00,0
2015-01-01 10:00:00,0
...,...
2016-01-01 02:00:00,0
2016-01-01 03:00:00,0
2016-01-01 04:00:00,0


In [83]:
# Scaling the data without predictos
X_data = datos.iloc[:, :4]
data_scaled = (X_data-X_data.mean())/(X_data.std())
data_scaled

,volt,rotate,pressure,vibration
datetime,,,,
2015-01-01 06:00:00,0.351562,-0.532940,1.142664,0.812209
2015-01-01 07:00:00,-0.519425,-0.834651,-0.479526,0.510211
2015-01-01 08:00:00,0.010187,1.551258,-2.341601,-1.156134
2015-01-01 09:00:00,-0.546614,-1.918403,0.790060,0.096684
2015-01-01 10:00:00,-0.863494,-0.209857,1.032971,-2.633602
...,...,...,...,...
2016-01-01 02:00:00,-1.117192,0.075019,-1.424538,0.561485
2016-01-01 03:00:00,0.103445,-1.352525,-0.469606,0.275228
2016-01-01 04:00:00,0.245145,0.271308,-0.477373,-1.315870


In [103]:
datosP = pd.concat([data_scaled, datos.iloc[:, [-1]]], axis=1)
datosN = datosP.to_numpy()
datosN = datosN.astype(np.float32)
datosN

array([[ 0.3515619 , -0.5329405 ,  1.1426637 ,  0.8122087 ,  0.        ],
       [-0.5194249 , -0.83465064, -0.47952616,  0.5102113 ,  0.        ],
       [ 0.01018681,  1.5512584 , -2.3416007 , -1.1561344 ,  0.        ],
       ...,
       [ 0.24514511,  0.27130783, -0.47737262, -1.3158696 ,  0.        ],
       [ 0.8743627 , -0.14386393,  0.1865564 ,  1.086801  ,  0.        ],
       [ 1.373824  , -1.2178224 ,  0.0207532 , -0.47744876,  0.        ]],
      dtype=float32)

In [104]:
def get_basic_model(input_dim, nb_classes):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense

    model = Sequential([
        Dense(16, input_shape=(input_dim,), activation='relu'),
        Dense(8, activation='relu'),
        Dense(4, activation='relu'),
        Dense(nb_classes, activation='sigmoid'),
    ])

    return model

model = get_basic_model(input_dim=4, nb_classes=1)
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 16)                80        
                                                                 
 dense_33 (Dense)            (None, 8)                 136       
                                                                 
 dense_34 (Dense)            (None, 4)                 36        
                                                                 
 dense_35 (Dense)            (None, 1)                 5         
                                                                 
Total params: 257 (1.00 KB)
Trainable params: 257 (1.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [105]:
# or instantiate an optimizer before passing it to model.compile
sgd = optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd)

tscv = TimeSeriesSplit()
for i, (train_index, test_index) in enumerate(tscv.split(datosN)):
    print(i, "TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test  = datosN[train_index,:-1],datosN[test_index,:-1]
    y_train, y_test  = datosN[train_index,[-1]],datosN[test_index,[-1]]
    model.fit(X_train, y_train, epochs=100, batch_size=4, validation_split=0, verbose=0)
    preds = model.predict(X_test, verbose=1)
    preds = np.argmax(preds,axis=1)
    preds=preds.reshape(1460,1)
    Accuracy = np.mean(preds == y_test)
    print('Accuracy = %.2f%s'%(Accuracy*100, '%'))


0 TRAIN: [   0    1    2 ... 1458 1459 1460] TEST: [1461 1462 1463 ... 2918 2919 2920]
46/46 [==============================] - 0s 2ms/step
Accuracy = 99.86%
1 TRAIN: [   0    1    2 ... 2918 2919 2920] TEST: [2921 2922 2923 ... 4378 4379 4380]
46/46 [==============================] - 0s 1ms/step
Accuracy = 99.93%
2 TRAIN: [   0    1    2 ... 4378 4379 4380] TEST: [4381 4382 4383 ... 5838 5839 5840]
46/46 [==============================] - 0s 1ms/step
Accuracy = 100.00%
3 TRAIN: [   0    1    2 ... 5838 5839 5840] TEST: [5841 5842 5843 ... 7298 7299 7300]
46/46 [==============================] - 0s 1ms/step
Accuracy = 99.86%
4 TRAIN: [   0    1    2 ... 7298 7299 7300] TEST: [7301 7302 7303 ... 8758 8759 8760]
46/46 [==============================] - 0s 1ms/step
Accuracy = 99.93%
